(singing-voice-extraction)=
# Singing voice extraction

TODO

In [ ]:
## Installing (if not) and importing compiam to the project
import importlib.util
if importlib.util.find_spec('compiam') is None:
    ## Bear in mind this will only run in a jupyter notebook / Collab session
    %pip install compiam
import compiam

# Import extras and supress warnings to keep the tutorial clean
import os
import numpy as np
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

Let's first print out the available tools we do have available to extract the pitch from Indian Art Music recordings.

In [ ]:
pprint(compiam.melody.pitch_extraction.list_tools())


## Leakage-aware source separation model

In [ ]:
%pip install tensorflow
%pip install tensorflow_addons

In [ ]:
# Importing and initializing a melodia instance
import soundfile as sf
from compiam import load_model, load_dataset
separation_model = load_model('separation:cold-diff-sep')

# Loading an normalizing an example track
saraga_carnatic = load_dataset(
    "saraga_carnatic",
    data_home=os.path.join("..", "audio", "mir_datasets")
)
saraga_tracks = saraga_carnatic.load_tracks()
example = saraga_tracks["109_Sri_Raghuvara_Sugunaalaya"]
input_mixture, sr = sf.read(example.audio_path)

input_mixture = input_mixture.T
mean = np.mean(input_mixture, keepdims=True)
std = np.std(input_mixture, keepdims=True)
input_mixture = (input_mixture - mean) / (1e-6 + std)

In [ ]:
### Getting 30 seconds and separating
input_mixture = input_mixture[:, :44100*30]
separation = separation_model.separate(
    input_data=input_mixture,
    input_sr=sr
)

In [ ]:
import IPython.display as ipd

# And we play it!
ipd.Audio(
    data=separation,
    rate=separation_model.sample_rate,
)